In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import pandas
import gensim

import matplotlib.pyplot as plt
import matplotlib

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import Ridge

import numpy as np



matplotlib.style.use('ggplot')

## 1. Read data

In [69]:
data_file = "../data/bdata.log.txt"

data = pandas.read_csv(data_file, sep = ',')

In [5]:
data.head()

,species,mhc,peptide_length,cv,sequence,inequality,meas
0,cow,BoLA-HD6,9,TBD,ALFYKDGKL,=,1.000000
1,cow,BoLA-HD6,9,TBD,ALYEKKLAL,=,1.000000
2,cow,BoLA-HD6,9,TBD,AMKDRFQPL,=,0.860543
3,cow,BoLA-HD6,9,TBD,AQRELFFTL,=,1.000000
4,cow,BoLA-HD6,9,TBD,FMKVKFEAL,=,0.957914


## 2. To one hot

In [25]:
selected_mhc = 'HLA-A*03:01'

indexes = data['mhc'][data['mhc'] == selected_mhc].index
indexes = data.iloc[indexes]['mhc'][data['peptide_length'] == 9].index
#indexes = data.index
selected_X = data['sequence'][indexes]
selected_y = data['meas'][indexes]

letters_X = selected_X.apply(list)

selected_X = pandas.DataFrame(list(letters_X))
selected_X = pandas.get_dummies(selected_X)

## 3. ml stuff

In [61]:
#normalize output
# m = selected_y.mean()
# d = selected_y.max() - selected_y.min()
# selected_y = (selected_y - m) / d

# train/test split
random_number = 43
X_train, X_test, y_train, y_test = train_test_split(selected_X, selected_y,
                                                    test_size = 0.33, random_state = random_number)

In [62]:
def ridge_regression(X, y, alpha):
    #Fit the model
    ridgereg = Ridge(alpha=alpha,normalize=True)
    ridgereg.fit(X, y)
    y_pred = ridgereg.predict(X)
    
    #Return the result in pre-defined format
    rss = sum((y_pred - y)**2)
    return rss, ridgereg.intercept_, ridgereg

In [63]:
alpha_ridge = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]

coef_matrix_ridge = pandas.DataFrame(index = ind, columns = col)
rss = [0] * 10
intercept = [0] * 10
predictors = [0] * 10

for i in range(10):
    rss[i], intercept[i], predictors[i] = ridge_regression(X_train, y_train, alpha_ridge[i])

In [64]:
# predict test data

test_rss = [0] * 10
for i in range(10):
    m_pred = predictors[i]
    test_pred = m_pred.predict(X_test)
    test_rss[i] = sum((test_pred - y_test)**2)
    
    print '-' * 10
    print "alpha = ", alpha_ridge[i]
    print "train rss = ", rss[i]
    print "test rss = ", test_rss[i]
    
m_pred = predictors[5]
test_pred = m_pred.predict(X_test)

----------
alpha =  1e-15
train rss =  90.6354700298
test rss =  47.2678783854
----------
alpha =  1e-10
train rss =  90.6354700298
test rss =  47.2678783844
----------
alpha =  1e-08
train rss =  90.6354700298
test rss =  47.2678782897
----------
alpha =  0.0001
train rss =  90.6354709683
test rss =  47.2669220363
----------
alpha =  0.001
train rss =  90.6355636929
test rss =  47.2583707688
----------
alpha =  0.01
train rss =  90.644655842
test rss =  47.178274906
----------
alpha =  1
train rss =  115.375670859
test rss =  55.4337456996
----------
alpha =  5
train rss =  174.212926702
test rss =  81.1123582844
----------
alpha =  10
train rss =  196.917880595
test rss =  91.2261006961
----------
alpha =  20
train rss =  212.319219506
test rss =  98.1086517388


In [65]:
test_pred = pandas.DataFrame(test_pred)

In [66]:
y_test = pandas.DataFrame(y_test)

In [67]:
test_pred['1'] = np.array(y_test)

In [68]:
test_pred.corr()

,0,1
0,1.000000,0.750098
1,0.750098,1.000000
